Welcome! This is a simple notebook to demo Small-Fry usage. If you have any questions, please feel free to contact [tginart](https://github.com/tginart).

For the purposes of this demo, we will be compressing 1,000 rows (words) from the offical [Wiki Gigawords GloVe embeddings]. Our demo embeddings are light-weight enough to keep within the code repository.  

Although Small-Fry can be used as a command line utility, it is recommended to use Small-Fry as an API. 

First, import the library:

In [1]:
import smallfry as sfry

Space constraints:
* We are going to use the default bitrate, R = 1, for compression.
* If you have more space for your embeddings, be sure to give yourself a looser memory constraint!
* You may specify either as an approximate memory budget (in bytes) or as a bitrate (avg bits per entry in embeddings matrix)
* For most downstream applications, in order to incur <1% loss in extrinsic performance, the bitrate should be somewhere in (0.1,3). See TODO:paperlink for more details.

Output directory:
* You may optionally specify an output directory using the param outdir. 
* Otherwise, Small-Fry will write to the same directory containing the source embeddings.

Prior:
* The prior should be specified as word frequency counts over a corpus. Small-Fry automatically normalizes the frequency counts into a probability vector. 
* The prior should be a Python dictionary mapping word to float, saved in the `.npy` format. See [`numpy.save`](https://docs.scipy.org/doc/numpy-1.14.0/re)
* Small-Fry is robust to noisy priors. If you do not have a prior of your own, and your application will be processing common English, please use [this prior], collected from the wiki16 corpus [cite].

Before making our call lets describe two variables we will need. Our `source_path` argument is 1000 lines out of the offical glove.6B.50d embeddings while our `prior_path` is a prior for these 1000 words in dict format saved as npy. We are now ready to make the API call:

In [2]:
word2idx, sfry_path = sfry.compress(sourcepath="data/glove.head.txt", priorpath="data/prior.npy")

[14, 29, 28, 14, 29, 29, 9, 31, 9, 3, 14, 16, 9, 9, 4, 29, 13, 13, 5, 3, 14, 16, 10, 7, 7, 21, 31, 7, 29, 3, 2, 7, 28, 9, 16, 28, 14, 16, 1, 7, 7, 16, 16, 13, 28, 14, 13, 31, 7, 16, 7, 14, 28, 1, 4, 10, 9, 18, 31, 3, 16, 13, 9, 9, 14, 16, 28, 16, 9, 28, 16, 10, 29, 28, 13, 21, 16, 7, 13, 3, 25, 28, 7, 28, 14, 31, 28, 29, 4, 1, 13, 4, 29, 9, 14, 1, 31, 18, 7, 29, 1, 14, 9, 16, 29, 13, 18, 28, 7, 5, 29, 28, 18, 16, 7, 7, 16, 28, 23, 28, 7, 31, 28, 3, 28, 12, 23, 16, 9, 28, 30, 28, 18, 31, 7, 7, 14, 28, 7, 13, 31, 28, 31, 13, 9, 14, 7, 28, 13, 23, 4, 10, 9, 14, 10, 4, 14, 18, 16, 28, 16, 31, 28, 23, 16, 28, 16, 7, 18, 7, 14, 28, 13, 31, 3, 12, 15, 3, 3, 14, 30, 28, 28, 5, 14, 7, 31, 13, 14, 1, 31, 29, 16, 13, 31, 14, 3, 3, 13, 23, 28, 24, 10, 29, 9, 3, 18, 27, 14, 1, 15, 9, 0, 11, 7, 7, 7, 18, 7, 29, 5, 14, 14, 7, 0, 19, 18, 29, 7, 7, 30, 10, 5, 22, 1, 31, 3, 1, 4, 16, 7, 8, 10, 16, 24, 16, 13, 31, 29, 16, 13, 24, 29, 18, 18, 5, 23, 5, 3, 10, 9, 24, 0, 10, 16, 7, 4, 0, 28, 29, 23, 13, 26,

Your Small-Fry embeddings have been written to file! I bet you they're pretty small! Let's check the filesizes before and after!

First, let's see how big the original embeddings are:

In [3]:
import os; print(str(os.path.getsize("data/glove.head.txt")) + " bytes")

428849 bytes


And now, let's check out the size of the Small-Fry embeddings:

In [4]:
print("Directory size: " + str(sfry.calc_folder_size(sfry_path)))

Directory size: 12830


Wow! That's small!

Now, let's see how we can efficiently query for word vectors without inflating the entire `.sfry` representation.

Like with other embeddings, the user is responsible for keeping track of the word representation returned by the Small-Fry compressor, in `word2idx`. Optionally, the word representation can be automatically saved by the `compress` call, using the `write-word-rep` flag.

The path to Small-Fry's compressed embeddings is returned in `sfry_path`. Both of these returns are used in Small-Fry's `query` API call.

We proceed to define the inputs for the querying:

We proceed to call the query routine on the word 'them':

In [ ]:
word_vector = sfry.query(word= "them", word2idx=word2idx, sfrypath=sfry_path) #returns a numpy vector

print(word_vector)

But reading from disk can be slow! That's why Small-Fry can create a memory-mapped representation using `numpy.memmap`.

Use `sfry.load` to generate a Python wrapper object as follows: 

In [ ]:
my_smallfry = sfry.load(sfry_path, word2idx) #generates a wrapper object for memory-mapped Small-Fry

And to query with your Small-Fry wrapper, just try:

In [ ]:
print(my_smallfry.query("them"))

It is as easy as that! You are now ready to use Small-Fry embeddings for your favorite NLP apps!